In [1]:
from fastai2.basics import *
from fastai2.vision.all import *
from faststyle import *

In [2]:
source = untar_data(URLs.COCO_SAMPLE)

In [3]:
stl_loss_mult = 3e5

In [ ]:
style_dir = Path('styles')

In [5]:
splitter = IncrementalSplitter()

In [6]:
def get_dl(bs, sz, path, pct=1., seed=None):
  splitter.pct = pct
  dblock = DataBlock(blocks=(ImageBlock, ImageBlock), get_items=get_image_files, splitter=splitter, 
                     item_tfms=[Resize(sz)], batch_tfms=[Normalize()])
  return dblock.dataloaders(path, bs=bs)

In [7]:
#TODO: hardcoded cuda
m = TransformerNet().cuda()

In [8]:
style_fns = L(style_dir/'starry_night.jpg', style_dir/'scream.jpg')

In [9]:
sname = '_'.join(style_fns.map(lambda x: x.stem))
model_name = f'{sname}-{stl_loss_mult:.0e}'
# szs_bss = [(256,22), (512,8), (1024,2), (1700, 1)]
szs_bss = [(256,11), (512,8), (1024,2), (1700, 1)]
lrs = [1e-3, 1e-3, 5e-4, 5e-5]
pcts = [1., .8, .6, .3]

In [10]:
import gc
# Need path on get_dl
def get_learner(sz, bs, pct=1., opt_func=Adam):
  learn = None; gc.collect() # Clear memory
  dl = get_dl(bs,sz,pct=pct,path=source)
  return style_learner(dl, style_fns).to_fp16()

In [11]:
i = 0

In [ ]:
for i in range(len(szs_bss)):
  learn = get_learner(*szs_bss[i], pct=pcts[i])
  if i!=0: learn.load(f'{model_name}-{szs_bss[i-1][0]}')
  learn.fit_one_cycle(1, lrs[i], pct_start=0.75)
  learn.save(f'{model_name}-{szs_bss[i][0]}')

epoch,train_loss,valid_loss,stl,cnt,tv,time
